<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/clear_deposit_rebalace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install -qqq web3

In [88]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math

In [89]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/demo"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
address = account.address

print("Connected:", w3.is_connected())
print("Your address:", address)

Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [90]:
def find_key(data, target_key):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            else:
                results.extend(find_key(value, target_key))
    elif isinstance(data, list):
        for item in data:
            results.extend(find_key(item, target_key))
    return results

In [91]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3.eth.get_balance(address)
  balances.append(["ETH", w3.from_wei(eth_balance, 'ether')])
  return balances

In [92]:
etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
CHAIN_ID = 11155111  # Sepolia

In [93]:
weth_address="0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"
usdc_address="0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238"
uni_address="0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984"
my_wallet_address = "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822"
eth_usdc_pool_address="0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1"

In [94]:
TOKEN_LIST = [weth_address,usdc_address,uni_address]

In [95]:
weth_address=w3.to_checksum_address(weth_address)
usdc_address=w3.to_checksum_address(usdc_address)
uni_address=w3.to_checksum_address(uni_address)
my_wallet_address=w3.to_checksum_address(my_wallet_address)
eth_usdc_pool_address=w3.to_checksum_address(eth_usdc_pool_address)

In [96]:
def fetch_abi(contract_address: str, api_key: str,inp_chain_id:int):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "contract",
        "action": "getabi",
        "address": contract_address,
        "apikey": api_key,
        "chainid":inp_chain_id
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if data.get("status") != "1":
        raise Exception(f"Error getting ABI: {data.get('message')}")
    abi = json.loads(data["result"])
    return abi

In [97]:
ERC20_ABI = fetch_abi(weth_address,etherscan_apikey,CHAIN_ID)
POOL_ABI = fetch_abi(eth_usdc_pool_address,etherscan_apikey,CHAIN_ID)

In [98]:
weth_contract = w3.eth.contract(address=weth_address, abi=ERC20_ABI)
usdc_contract = w3.eth.contract(address=usdc_address, abi=ERC20_ABI)
pool_contract = w3.eth.contract(address=eth_usdc_pool_address, abi=POOL_ABI)

In [99]:
def pool_current_info(inp_pool_contract):
  token0_address = inp_pool_contract.functions.token0().call()
  token1_address = inp_pool_contract.functions.token1().call()
  slot0 = inp_pool_contract.functions.slot0().call()
  liquidity = inp_pool_contract.functions.liquidity().call()
  fee = inp_pool_contract.functions.fee().call()
  tickspacing = inp_pool_contract.functions.tickSpacing().call()
  return {'token0_address':token0_address,'token1_address':token1_address,'sqrtprice':slot0[0],'current_tick':slot0[1],'liquidity':liquidity,'fee_tier':fee,'tick_spacing':tickspacing}

In [100]:
pool_info=pool_current_info(pool_contract)
pool_info

{'token0_address': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1_address': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'sqrtprice': 889998938386158993659925974409017,
 'current_tick': 186542,
 'liquidity': 11478317700026758,
 'fee_tier': 500,
 'tick_spacing': 10}

In [101]:
def calculate_price_by_tick(tick:int,decimal_0:int,decimal_1:int):
  return (10**(decimal_0-decimal_1))*(1.0001**tick)

In [102]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [103]:
token0_contract = w3.eth.contract(address=pool_info['token0_address'], abi=ERC20_ABI)
token1_contract = w3.eth.contract(address=pool_info['token1_address'], abi=ERC20_ABI)

In [104]:
calculate_price_by_tick(pool_info['current_tick'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())

0.00012618597290535495

In [105]:
true_price=calculate_price_from_sqrt_price(pool_info['sqrtprice'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())
true_price

0.00012618850923876184

In [106]:
#eth_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd").json()["ethereum"]["usd"]
eth_sepolia_price=1/calculate_price_from_sqrt_price(pool_info['sqrtprice'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())
eth_sepolia_price

7924.651824738618

In [107]:
POSITION_MANAGER_ADDRESS = "0x1238536071E1c677A632429e3655c799b22cDA52"
POSITION_MANAGER_ADDRESS = w3.to_checksum_address(POSITION_MANAGER_ADDRESS)
POSITION_MANAGER_ABI = fetch_abi(POSITION_MANAGER_ADDRESS,etherscan_apikey,CHAIN_ID)
POSITION_MANAGER_contract = w3.eth.contract(address=POSITION_MANAGER_ADDRESS, abi=POSITION_MANAGER_ABI)

In [108]:
def check_liquidity_nft(inp_address,inp_postion_manger_address,inp_position_manager_abi):
  pm_contract=w3.eth.contract(address=inp_postion_manger_address, abi=inp_position_manager_abi)
  nfts=[]
  try:
    balance = pm_contract.functions.balanceOf(address).call()
    print(f"تعداد NFT پوزیشن‌ها: {balance}")


    if balance > 0:
        for i in range(balance):

            token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()

            pos = pm_contract.functions.positions(token_id).call()
            dct={'token_id':token_id,'token0':pos[0],'token1':pos[3],'fee_tier':pos[4],'liquidity':pos[7],'tick_range':[pos[5],pos[6]]}
            nfts.append(dct)
        return nfts
    else:
        print("lp nft not found")

  except Exception as e:
    print(f"error in read data --> {e}")

In [109]:
check_liquidity_nft(my_wallet_address,POSITION_MANAGER_ADDRESS,POSITION_MANAGER_ABI)

تعداد NFT پوزیشن‌ها: 4


[{'token_id': 217011,
  'token0': 0,
  'token1': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
  'fee_tier': 500,
  'liquidity': 0,
  'tick_range': [142500, 156410]},
 {'token_id': 220887,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 1824073543371,
  'tick_range': [184800, 186800]},
 {'token_id': 220890,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 15531994169743,
  'tick_range': [185000, 187000]},
 {'token_id': 220931,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 1581033524365,
  'tick_range': [185170, 187400]}]

In [110]:
v3factoryaddress="0x0227628f3F023bb0B980b67D528571c95c6DaC1c"

In [111]:
def get_v3_pool_address(inp_token0_address,inp_token1_address,inp_fee_tier,inp_factory_address,inp_scan_apikey,inp_chainid):
  FACTORY_ADDRESS = w3.to_checksum_address(v3factoryaddress)
  FACTORY_ABI =fetch_abi(inp_factory_address,etherscan_apikey,CHAIN_ID)
  factory = w3.eth.contract(address=FACTORY_ADDRESS, abi=FACTORY_ABI)
  pool_address_v = factory.functions.getPool(inp_token0_address, inp_token1_address, inp_fee_tier).call()
  return pool_address_v

In [112]:
get_v3_pool_address(usdc_address,weth_address,500,v3factoryaddress,etherscan_apikey,CHAIN_ID)

'0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1'

In [113]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [114]:
def calculate_sqrt_price_from_price(price: float, decimal_0: int, decimal_1: int) -> float:
  return (2**96)*((price/(10**(decimal_1-decimal_0)))**(0.5))

In [115]:
if pool_info['sqrtprice']>0:
  a=calculate_price_from_sqrt_price(pool_info['sqrtprice'],token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())
  print(a)
  b=calculate_sqrt_price_from_price(a,token0_contract.functions.decimals().call(),token1_contract.functions.decimals().call())
  print(b)
  print(pool_info['sqrtprice'])

0.00012618850923876184
8.89998938386159e+20
889998938386158993659925974409017


In [116]:
def init_calculate_Liquid_amounts(lower,upper,capital):
  init_price=(lower+upper)/2
  s=np.sqrt(init_price)
  su=np.sqrt(upper)
  sl=np.sqrt(lower)
  amount_0=(capital/2)/init_price
  amount_1=capital/2
  l_0=amount_0*s*su/(su-s)
  l_1=amount_1/(s-sl)
  l=min(l_0,l_1)
  return l,amount_0,amount_1

In [117]:
def init_amounts_liquid(inp_current_tick: int, inp_tickspacing: int, step: int,inp_sqrt_price,
                       decimal0, decimal1, token0_real_price: float,
                       token1_real_price: float, inp_hypo_capital: int):

    tick_range = [inp_current_tick - (step * inp_tickspacing),
                  inp_current_tick + (step * inp_tickspacing)]

    price_low = calculate_price_by_tick(tick_range[0], decimal0, decimal1)
    price_high = calculate_price_by_tick(tick_range[1], decimal0, decimal1)

    current_price = calculate_price_from_sqrt_price(inp_sqrt_price, decimal0, decimal1)
    #current_price = (price_low+price_high)/2
    s = np.sqrt(current_price)
    su = np.sqrt(price_high)
    sl = np.sqrt(price_low)

    amount_0 = (inp_hypo_capital / 2.0) / token0_real_price
    amount_1 = (inp_hypo_capital / 2.0) / token1_real_price

    if su == s:
        l_0 = np.inf
    else:
        l_0 = amount_0 * s * su / (su - s)

    if s == sl:
        l_1 = np.inf
    else:
        l_1 = amount_1 / (s - sl)

    l = min(l_0, l_1)

    final_amount_0 = l * (su - s) / (s * su)
    final_amount_1 = l * (s - sl)

    return l, final_amount_0, final_amount_1

In [118]:
def calculate_new_amount_token_mix(current_price: float, lower: float, upper: float, liquidity: float):
    s = np.sqrt(current_price)
    su = np.sqrt(upper)
    sl = np.sqrt(lower)

    amount_0 = 0.0
    amount_1 = 0.0

    if current_price < lower:
        amount_0 = liquidity * ((1.0 / sl) - (1.0 / su))
        amount_1 = 0.0
    elif current_price > upper:
        amount_0 = 0.0
        amount_1 = liquidity * (su - sl)
    else:
        amount_0 = liquidity * ((1.0 / s) - (1.0 / su))
        amount_1 = liquidity * (s - sl)

    return amount_0, amount_1

In [119]:
#token0:usdc,token1:weth

In [120]:
decimal_0=token0_contract.functions.decimals().call()
decimal_1=token1_contract.functions.decimals().call()

In [121]:
stepv=10

In [122]:
init_amounts_liquid(pool_info['current_tick'],pool_info['tick_spacing'],stepv,pool_info['sqrtprice'],decimal_0,decimal_1,1,eth_sepolia_price,100)

(np.float64(112.3949865681931),
 np.float64(49.79989563240644),
 np.float64(0.0063094254619380915))

In [123]:
lowerv=calculate_price_by_tick(pool_info['current_tick']-(stepv*pool_info['tick_spacing']),decimal_0,decimal_1)
upperv=calculate_price_by_tick(pool_info['current_tick']+(stepv*pool_info['tick_spacing']),decimal_0,decimal_1)

In [124]:
print(lowerv,upperv)

0.00012493046395747616 0.0001274540992939145


In [125]:
L=init_amounts_liquid(pool_info['current_tick'],pool_info['tick_spacing'],stepv,pool_info['sqrtprice'],decimal_0,decimal_1,1,eth_sepolia_price,100)
L

(np.float64(112.3949865681931),
 np.float64(49.79989563240644),
 np.float64(0.0063094254619380915))

In [126]:
calculate_new_amount_token_mix(0.0001265 ,lowerv,upperv,L[0])

(np.float64(37.47369364248848), np.float64(0.00786676908139827))

In [127]:
approve_tx0 = token0_contract.functions.approve(POSITION_MANAGER_ADDRESS, 2**256-1).build_transaction({
    "from": my_wallet_address,
    "nonce": w3.eth.get_transaction_count(my_wallet_address)+1
})
# sign + send

approve_tx1 = token1_contract.functions.approve(POSITION_MANAGER_ADDRESS, 2**256-1).build_transaction({
    "from": my_wallet_address,
    "nonce": w3.eth.get_transaction_count(my_wallet_address)+1
})
# sign + send
print(approve_tx0)
print(approve_tx1)

{'value': 0, 'gas': 39052, 'maxPriorityFeePerGas': 1000000, 'maxFeePerGas': 1000584, 'chainId': 11155111, 'from': '0x7868EDEfC36A74eba220C75aCFB05c59a4e55822', 'nonce': 77, 'to': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 'data': '0x095ea7b30000000000000000000000001238536071e1c677a632429e3655c799b22cda52ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'}
{'value': 0, 'gas': 26807, 'maxPriorityFeePerGas': 1000000, 'maxFeePerGas': 1000584, 'chainId': 11155111, 'from': '0x7868EDEfC36A74eba220C75aCFB05c59a4e55822', 'nonce': 77, 'to': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'data': '0x095ea7b30000000000000000000000001238536071e1c677a632429e3655c799b22cda52ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'}


In [128]:
portfolio=check_token_balance(TOKEN_LIST,my_wallet_address)
portfolio

[['WETH', 0.03764500076879961],
 ['USDC', 64.902607],
 ['UNI', 0.000830846866961009],
 ['ETH', Decimal('0.15572140252649461')]]

In [129]:
init_amount1=portfolio[0][1]/10
init_amount0=init_amount1*eth_sepolia_price
print(init_amount0)
print(init_amount1)

29.832352403475454
0.003764500076879961


In [130]:
LP=init_amounts_liquid(pool_info['current_tick'],pool_info['tick_spacing'],stepv,pool_info['sqrtprice'],decimal_0,decimal_1,1,eth_sepolia_price,2*init_amount0)

In [131]:
LP

(np.float64(67.06013695372454),
 np.float64(29.712960723244944),
 np.float64(0.003764500076879961))

In [132]:
req_amount0,req_amount1=calculate_new_amount_token_mix(true_price,lowerv,upperv,LP[0])

In [133]:
print(f"--- Checking Token 0 ---")
balance0 = token0_contract.functions.balanceOf(my_wallet_address).call()
allowance0 = token0_contract.functions.allowance(my_wallet_address, POSITION_MANAGER_ADDRESS).call()

print(f"Required:  {req_amount0}")
print(f"Balance:   {balance0}   -> {'✅ OK' if balance0 >= req_amount0 else '❌ INSUFFICIENT BALANCE'}")
print(f"Allowance: {allowance0} -> {'✅ OK' if allowance0 >= req_amount0 else '❌ NOT APPROVED YET'}")

print(f"\n--- Checking Token 1 ---")
balance1 = token1_contract.functions.balanceOf(my_wallet_address).call()
allowance1 = token1_contract.functions.allowance(my_wallet_address, POSITION_MANAGER_ADDRESS).call()

print(f"Required:  {req_amount1}")
print(f"Balance:   {balance1}   -> {'✅ OK' if balance1 >= req_amount1 else '❌ INSUFFICIENT BALANCE'}")
print(f"Allowance: {allowance1} -> {'✅ OK' if allowance1 >= req_amount1 else '❌ NOT APPROVED YET'}")

--- Checking Token 0 ---
Required:  29.712960723244667
Balance:   64902607   -> ✅ OK
Allowance: 115792089237316195423570985008687907853269984665640564039457584007912734772744 -> ✅ OK

--- Checking Token 1 ---
Required:  0.003764500076879961
Balance:   37645000768799609   -> ✅ OK
Allowance: 115792089237316195423570985008687907853269984665640564039457584007913129639935 -> ✅ OK


In [134]:
amount_to_approve = 2**256 - 1  # Infinite approval

print("🚀 Sending Approve Transaction for Token 1...")

# 1. گرفتن Nonce دقیق (شامل تراکنش‌های پندینگ)
nonce = w3.eth.get_transaction_count(my_wallet_address, 'pending')

# 2. ساخت تراکنش
tx = token1_contract.functions.approve(POSITION_MANAGER_ADDRESS, amount_to_approve).build_transaction({
    'from': my_wallet_address,
    'nonce': nonce,
    'gasPrice': w3.eth.gas_price  # اختیاری: برای شبکه هایی مثل پالیگان لازم است
})

# 3. امضا و ارسال
signed_tx = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print(f"⏳ Transaction sent! Hash: {w3.to_hex(tx_hash)}")
print("Waiting for confirmation...")

# 4. صبر برای تایید (مهمترین بخش)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

if tx_receipt.status == 1:
    print("✅ Token 1 Approved Successfully!")
    print("👉 Now you can run your MINT function.")
else:
    print("❌ Approve Transaction Failed on-chain.")

🚀 Sending Approve Transaction for Token 1...
⏳ Transaction sent! Hash: 0xb37f0395c23fea326519eb7a7d77336097ecf6607b1f7dc41350897c728d05ff
Waiting for confirmation...
✅ Token 1 Approved Successfully!
👉 Now you can run your MINT function.


In [137]:
def get_nearest_valid_tick(tick, spacing):
    return (tick // spacing) * spacing

valid_current_tick = get_nearest_valid_tick(pool_info['current_tick'], pool_info['tick_spacing'])

tick_lower = valid_current_tick - (stepv * pool_info['tick_spacing'])
tick_upper = valid_current_tick + (stepv * pool_info['tick_spacing'])

print(f"Original Tick: {pool_info['current_tick']}")
print(f"Valid Lower: {tick_lower} (Divisible by 10? {tick_lower % 10 == 0})")
print(f"Valid Upper: {tick_upper} (Divisible by 10? {tick_upper % 10 == 0})")

# ----------------------------------------------

pm_contract = w3.eth.contract(address=POSITION_MANAGER_ADDRESS, abi=POSITION_MANAGER_ABI)

params = {
    "token0": usdc_address,
    "token1": weth_address,
    "fee": pool_info['fee_tier'],
    "tickLower": tick_lower,
    "tickUpper": tick_upper,
    "amount0Desired": int(req_amount0 * 10**decimal_0),
    "amount1Desired": w3.to_wei(req_amount1, "ether"),
    "amount0Min": 0,
    "amount1Min": 0,
    "recipient": my_wallet_address,
    "deadline": int(time.time()) + 1200
}

# ساخت تراکنش
tx = pm_contract.functions.mint(params).build_transaction({
    "from": my_wallet_address,
    "nonce": w3.eth.get_transaction_count(my_wallet_address),
    "gas": 600000,
    "gasPrice": w3.eth.gas_price
})

signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print("TX SENT:", w3.to_hex(tx_hash))

Original Tick: 186542
Valid Lower: 186440 (Divisible by 10? True)
Valid Upper: 186640 (Divisible by 10? True)
TX SENT: 0x7eeec97b2f714232a1378646b229f100385ecf2eaf5f950036ef7f0112d3d2d5


In [138]:
check_token_balance(TOKEN_LIST,my_wallet_address)

[['WETH', 0.030116000615040158],
 ['USDC', 52.680195],
 ['UNI', 0.000830846866961009],
 ['ETH', Decimal('0.154471901902105642')]]

In [146]:
LP_NFTS=check_liquidity_nft(my_wallet_address,POSITION_MANAGER_ADDRESS,POSITION_MANAGER_ABI)

تعداد NFT پوزیشن‌ها: 6


In [147]:
LP_NFTS

[{'token_id': 217011,
  'token0': 0,
  'token1': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
  'fee_tier': 500,
  'liquidity': 0,
  'tick_range': [142500, 156410]},
 {'token_id': 220887,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 1824073543371,
  'tick_range': [184800, 186800]},
 {'token_id': 220890,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 15531994169743,
  'tick_range': [185000, 187000]},
 {'token_id': 220931,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 1581033524365,
  'tick_range': [185170, 187400]},
 {'token_id': 221423,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 10000,
  'liquidity': 4876927719795,
  'tick_range': [185000, 187000]},
 {'token_id': 221424,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 5

In [148]:
if not w3.is_connected():
    exit()

# 1. خواندن نانس اولیه یکبار قبل از شروع حلقه
current_nonce = w3.eth.get_transaction_count(my_wallet_address)

for nft in LP_NFTS[:-1]:
    POSITION_TOKEN_ID = nft['token_id']
    liquidity_amount = nft['liquidity']

    # --- Burn پوزیشن های خالی (Liquidity == 0) ---
    if liquidity_amount == 0:
        # استفاده از build_transaction و sign/send برای مدیریت نانس
        burn_tx = pm_contract.functions.burn(POSITION_TOKEN_ID).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": 400000,
            "gasPrice": w3.eth.gas_price
        })

        signed_burn_tx = w3.eth.account.sign_transaction(burn_tx, PRIVATE_KEY)
        hash_burn = w3.eth.send_raw_transaction(signed_burn_tx.raw_transaction)
        print(f"Burn TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_burn)}")

        # ⚠️ افزایش نانس محلی
        current_nonce += 1
        time.sleep(10)
    # --- Decrease Liquidity و Collect و Burn پوزیشن های فعال (Liquidity > 0) ---
    else:
        # A. Decrease Liquidity
        params_decrease = {
            "tokenId": POSITION_TOKEN_ID,
            "liquidity": liquidity_amount,
            "amount0Min": 0,
            "amount1Min": 0,
            "deadline": int(time.time()) + 1200
        }

        decrease_tx = pm_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": 400000,
            "gasPrice": w3.eth.gas_price
        })

        signed_decrease_tx = w3.eth.account.sign_transaction(decrease_tx, PRIVATE_KEY)
        hash_decrease = w3.eth.send_raw_transaction(signed_decrease_tx.raw_transaction)
        print(f"Decrease TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_decrease)}")

        # ⚠️ افزایش نانس محلی
        current_nonce += 1

        # B. Collect Fees/Tokens
        MAX_UINT128 = 2**128 - 1
        params_collect = {
            "tokenId": POSITION_TOKEN_ID,
            "recipient": my_wallet_address,
            "amount0Max": MAX_UINT128,
            "amount1Max": MAX_UINT128
        }

        collect_tx = pm_contract.functions.collect(params_collect).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": 400000,
            "gasPrice": w3.eth.gas_price
        })

        signed_collect_tx = w3.eth.account.sign_transaction(collect_tx, PRIVATE_KEY)
        hash_collect = w3.eth.send_raw_transaction(signed_collect_tx.raw_transaction)
        print(f"Collect TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_collect)}")

        # ⚠️ افزایش نانس محلی
        current_nonce += 1

        # C. Burn NFT
        burn_tx_active = pm_contract.functions.burn(POSITION_TOKEN_ID).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": 400000,
            "gasPrice": w3.eth.gas_price
        })

        signed_burn_tx_active = w3.eth.account.sign_transaction(burn_tx_active, PRIVATE_KEY)
        hash_burn_active = w3.eth.send_raw_transaction(signed_burn_tx_active.raw_transaction)
        print(f"Burn TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_burn_active)}")

        # ⚠️ افزایش نانس محلی
        current_nonce += 1
        time.sleep(10)

Burn TX SENT for ID 217011: 0x7867b42cd13ac1801620f899d71f62ea0fa1ec4f0bbbdb2a5e077baece705a46
Decrease TX SENT for ID 220887: 0xb1a053675104edaef1dad278b2f0ec36cf88f334e1f872181dc8910a2cd621f8
Collect TX SENT for ID 220887: 0x822c6271ce07b8f6a69a40307cf3b9b94db15d8ec967660eed2606b4f8995254
Burn TX SENT for ID 220887: 0xef3e98cbf734ec99739f97358af4db5fdae4f9fa49f3684bdc1c038c13d09f50
Decrease TX SENT for ID 220890: 0x04dcea0d23dc54d5b750e916943beaa250f7e784edb33cb87b01f0481ea32bbf
Collect TX SENT for ID 220890: 0xafee4ff5bbf8b6f10957c486ccc0506c455790155e6e37e77c00225b9b544c04
Burn TX SENT for ID 220890: 0x225140a62f2c2beb67de4996af7919c232ebf71c600e415697e4ac42dbba42c6
Decrease TX SENT for ID 220931: 0xba0cf374a099b0c9c50877db4752ae2da4c7eff1cd759902cf06d815bebc88c3
Collect TX SENT for ID 220931: 0x769a138127a5bd39d4df3a72ee9ff3cbb7ef4366ddffc5b472e903c946f1597e
Burn TX SENT for ID 220931: 0x17b182913a7f84054273e234bd40388962877073cd30c759d30ee44f00eee905
Decrease TX SENT for ID 22142

In [149]:
check_liquidity_nft(my_wallet_address,POSITION_MANAGER_ADDRESS,POSITION_MANAGER_ABI)

تعداد NFT پوزیشن‌ها: 1


[{'token_id': 221424,
  'token0': 0,
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 33516997100150,
  'tick_range': [186440, 186640]}]

In [150]:
check_token_balance(TOKEN_LIST,my_wallet_address)

[['WETH', 0.05731403129150014],
 ['USDC', 59.170609],
 ['UNI', 0.000830846866961009],
 ['ETH', Decimal('0.154470428261869894')]]